<a href="https://colab.research.google.com/github/Aditya6122/HashtagifyMe/blob/main/HashtagGenerator_HashtagifyMe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers --quiet
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.9 MB/s eta 0:00:00


In [ ]:
import torch
import datasets
from datasets import load_dataset
import requests
from PIL import Image
from transformers import BlipForConditionalGeneration, AutoProcessor

In [ ]:
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

In [ ]:
dataset_name = "conceptual_captions"
labeled_config = "labeled"
dataset = load_dataset(dataset_name,labeled_config,split='train')
dataset = dataset.train_test_split(test_size=0.10, shuffle=True, seed=42)
dataset['train'] = dataset['train'].train_test_split(test_size=0.12, shuffle=True, seed=42)
data = {}
data['train'] = dataset['train']['train']
data['eval'] = dataset['train']['test']
data['test'] = dataset['test']
data['train'].set_format("torch")
data['eval'].set_format("torch")
data['test'].set_format("torch")

Generating train split:   0%|          | 0/2007090 [00:00<?, ? examples/s]

Dataset conceptual_captions downloaded and prepared to /root/.cache/huggingface/datasets/conceptual_captions/labeled/1.0.0/05266784888422e36944016874c44639bccb39069c2227435168ad8b02d600d8. Subsequent calls will reuse this data.


In [ ]:
import io
import urllib
import PIL.Image
from datasets.utils.file_utils import get_datasets_user_agent

In [ ]:
USER_AGENT = get_datasets_user_agent()
def fetch_single_image(image_url, timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers={"user-agent": USER_AGENT},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
            break
        except Exception:
            image = None
    return image

In [ ]:
from torch.utils.data import Dataset
import io
import urllib
import PIL.Image
from datasets.utils.file_utils import get_datasets_user_agent

class ConceptualCaptionCustom(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor
        USER_AGENT = get_datasets_user_agent()

    def __getitem__(self, idx):
        item = self.dataset[idx]
        img_url = item['image_url']
        label = item['labels']
        img = fetch_single_image(img_url)

        if img is None:
            return

        img = self.processor.image_processor(img, return_tensors="pt")
        target = [label[0]]
        for i in label[1:]:
            target.append('[SEP]')
            target.append(i)

        target = ' '.join([i for i in target])
        target = self.processor.tokenizer.encode(target,return_tensors='pt')

        return {'image':img['pixel_values'],'target':target,'sequence_labels':label}

    def __len__(self):
        return len(self.dataset)

In [ ]:
train_dataset = ConceptualCaptionCustom(data['train'], processor)
val_dataset = ConceptualCaptionCustom(data['eval'], processor)
test_dataset = ConceptualCaptionCustom(data['test'],processor)

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    imgs = []
    targets = []
    for i in batch:
        if(i is not None):
            imgs.append(i['image'])
            targets.append(i['target'][0])

    targets = pad_sequence(targets).permute(1,0)
    imgs = torch.stack(imgs)

    return imgs,targets

In [ ]:
from torch.utils.data import DataLoader

train_dataloader =  DataLoader(train_dataset, batch_size=4, collate_fn = collate_fn, num_workers=4)
val_dataloader =  DataLoader(val_dataset, batch_size=2, collate_fn = collate_fn, num_workers=2)
test_dataloader =  DataLoader(test_dataset, batch_size=2, collate_fn = collate_fn, num_workers=2)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import torch.nn as nn

In [ ]:
params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.SGD(params, lr=1e-1, momentum=0.09,weight_decay=0.000005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=5, gamma=0.1)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 5

In [ ]:
a = next(iter(train_dataloader))[0][1]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
url = 'http://www.hawaiigolf.com/photo-galleries/images/preview/28848.jpg'
raw_image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(raw_image, return_tensors="pt").to("cuda")
model.eval()
with torch.no_grad():
    for i in range(5):
        out = model.generate(**inputs,do_sample=True, num_beams=4, max_length=30)
        caption = processor.decode(out[0], skip_special_tokens=True)
        print(caption)

lush green golf course with mountains in the background and clouds in the sky
green grass in a golf course with a mountain in the background
trees and bushes surround a green golf course with a mountain in the background
this is a view of a golf course with a mountain in the background
dirt path leading to the green of a golf course with a mountain in the background


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib

import PIL.Image

from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent


USER_AGENT = get_datasets_user_agent()


def fetch_single_image(image_url, timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers={"user-agent": USER_AGENT},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
            break
        except Exception:
            image = None
    return image


def fetch_images(urls, num_threads, timeout=None, retries=0):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        imgs = list(executor.map(fetch_single_image_with_args, urls))
    return batch